TODO:
- invastigate models learning "static model"
- slow ADAM run, does use_gpu work?

# Data loading

In [ ]:
import numpy as np
from IPython.display import display, Image

from platform import python_version
print(python_version())

In [ ]:
# data is in float64 format: 0.0-0.000255
def load_data(type='training', region=1):
    # `raw_validation_set.npy` is multiple tries non-averaged `validation_set.npy`
    return (
        (np.load(f'./Data/region{region}/{type}_inputs.npy')*1_000_000).astype(np.uint8),
        np.load(f'./Data/region{region}/{type}_set.npy')
    )
    
(input_tr, output_tr) = load_data('training', 1) 
(val_input, val_output) = load_data('validation', 1)

# Data exploration

In [ ]:
display(input_tr.shape, output_tr.shape)
display(val_input.shape, val_output.shape)

In [ ]:
display(input_tr.shape)
display(input_tr[0])
display(input_tr.dtype)
display(min(input_tr[0]), max(input_tr[0]), np.mean(input_tr[0]), np.std(input_tr[0]))
display(min(input_tr[1]), max(input_tr[1]), np.mean(input_tr[1]), np.std(input_tr[1]))

display(output_tr.shape)
display(output_tr[0])
display(output_tr.dtype)
display(min(output_tr[0]), max(output_tr[0]), np.mean(output_tr[0]), np.std(output_tr[0]))


In [ ]:
from PIL import Image

def reshape_single_as_picture(input, size=31):
    return np.reshape(input, (size, size))

def as_single_picture(input, size=31):
    return Image.fromarray(reshape_single_as_picture(input, size), 'L')

In [ ]:
display(as_single_picture(input_tr[0]))
display(as_single_picture(input_tr[2]))
display(as_single_picture(input_tr[700]))
display(as_single_picture(input_tr[-1]))
display(as_single_picture(input_tr[-2]))

display(input_tr[0]*1_000_000)

# Data trasnform

In [ ]:
def downsample_input(dta):
    def process_single(pic):
        resized_pic = as_single_picture(pic).resize(size=(15, 15), resample=Image.BICUBIC)
        return np.array(resized_pic).reshape(15*15)
    return np.array([process_single(pic) for pic in dta])

def normalize_input(dta):
    return (dta/np.std(dta))

In [ ]:
input_tr_processed = downsample_input(input_tr)
display(as_single_picture(input_tr_processed[0], 15))
display(as_single_picture(input_tr_processed[2], 15))
display(as_single_picture(input_tr_processed[-1], 15))
display(as_single_picture(input_tr_processed[-2], 15))

display(input_tr_processed[0], min(input_tr_processed[0]), max(input_tr_processed[0]))

# Network parameters

In [ ]:
_, output_shape = output_tr.shape
dta_len, input_shape = input_tr_processed.shape
display(dta_len, output_shape, input_shape)

In [ ]:
import NDN3.NDNutils as NDNutils
import NDN3.NDN as NDN

from datetime import datetime

In [ ]:
hls = 40
d2x = 0.0005
l1 = 0.000001

hsm_params = NDNutils.ffnetwork_params(
    input_dims=[1, input_shape], 
    layer_sizes=[hls, 2*hls, output_shape],
    ei_layers=[0, hls // 2],
    normalization=[0], 
    layer_types=['normal','normal','normal'], 
    reg_list={
        'd2x':[d2x,None,None],
        'l1':[l1,None,None],
        'max':[None,None,100]})
hsm_params['weights_initializers']=['normal','normal','normal']
hsm_params['normalize_weights']=[1,0,0]

display(hsm_params)

# Network training

In [ ]:
time_str = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

hsm = NDN.NDN(hsm_params, noise_dist='poisson')
hsm.train(
    input_data=input_tr_processed, 
    output_data=output_tr, 
    train_indxs=np.array(range(dta_len)), 
    test_indxs=np.array(range(dta_len)), 
    learning_alg="lbfgs", 
    opt_params={
        'batch_size': 2, 'use_gpu': False, 'epochs_summary': 10, 'epochs_training': 700
    }, 
    output_dir=f"logs/{time_str}"
)

In [ ]:
results = hsm.generate_prediction(input_tr_processed)
display(results[0])
display(output_tr[0])

# Results evaluation

In [ ]:
import scipy.stats

def get_correlation(a, b):
    assert a.shape == b.shape
    c = [scipy.stats.pearsonr(a[:,i], b[:,i])[0] for i in range(a.shape[1])]
    return np.array(c)

In [ ]:
get_correlation(output_tr, results)

In [ ]:
import matplotlib.pyplot as plt  # plotting

input_size = [15, 15]
nrows, ncols = 3, 4

fig, _ = plt.subplots(nrows=nrows, ncols=ncols)
fig.set_size_inches(12, 6)
for neuron_i in range(nrows * ncols):
    plt.subplot(nrows, ncols, neuron_i+1)
    w = hsm.networks[0].layers[0].weights[:,neuron_i]
    plt.imshow(np.reshape(w, input_size), cmap='Greys', interpolation='none', vmin=-max(abs(w)), vmax=max(abs(w)))
plt.show()

hsm.networks[0].layers[0].weights.shape

In [ ]:
eval_info = hsm.eval_models(input_data=input_tr_processed, output_data=output_tr, data_indxs=np.array(range(dta_len)), nulladjusted=True)
eval_info

In [ ]:
display(np.var(results, axis=0))
display(np.var(output_tr, axis=0))

In [ ]:
time_str = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

hsm = NDN.NDN(hsm_params, noise_dist='poisson')
hsm.train(
    input_data=input_tr_processed, 
    output_data=output_tr, 
    train_indxs=np.array(range(dta_len)), 
    test_indxs=np.array(range(dta_len)), 
    learning_alg="adam", 
    opt_params={
        'batch_size': 2, 'use_gpu': False, 'epochs_summary': 10, 'epochs_training': 700
    }, 
    output_dir=f"logs/{time_str}"
)
eval_info = hsm.eval_models(input_data=input_tr_processed, output_data=output_tr, data_indxs=np.array(range(dta_len)), nulladjusted=True)
eval_info

In [ ]:
results = hsm.generate_prediction(input_tr_processed)
display(results[0])
display(output_tr[0])
display(np.mean(output_tr, axis=0))

eval_info = hsm.eval_models(input_data=input_tr_processed, output_data=output_tr, data_indxs=np.array(range(dta_len)), nulladjusted=True)
eval_info